In [26]:

#----------ESSENTIAL_IMPORTS-----------------
from config import *
from model import *
from utils import *
from DopeTech import *
#--------------------------------------------



In [ ]:


def mainCycle():
    """
        Initialising two nets
        first one - key net, which we are training
        second one - net, that we use to estimate Q-value-function
        for next states for Bellman Equation
        """
    #--------------------------------------------
    #keyNet = QModel().to(DEVICE)
    keyNet = loadFromFile("QNet.pkl")
    helperNet = QModel().to(DEVICE)
    helperNet.load_state_dict(keyNet.state_dict())
    helperNet.eval()
    #--------------------------------------------
    optimizer = optim.Adam(keyNet.parameters(), lr = 1e-4)
    gameMemory = GameMemory(50000)
    ENVIRONMENT.render(mode = 'rgb_array')
    
    
    #fillGameMemoryWithRandomTransitions(gameMemory)
    #saveToFile(gameMemory, "gameMemory.pkl")
    gameMemory = loadFromFile("gameMemory.pkl")
    stepsDone = 0
    normalAction = lambda state: keyNet(state).max(1)[1].view(1, 1)
    stateHolder = OneStateHolder()
    
    
    print("started learning")
    for e in range(AMOUNT_OF_EPISODES):
        #progressBar = myProgressBar(5)
        currentLifes = 5
        pureRewardPerGame = 0
        totalDisqountedReward = 0
        ENVIRONMENT.reset()
        stateHolder.initWithFirstScreens()
        isDone = False
        while not isDone:
            #performing action choosing according to epsilon greedy rule
            ENVIRONMENT.render(mode = "rgb_array")
            action = epsilonGreedyChooser(normalAction, stateHolder.getState().unsqueeze(0), stepsDone)
            stepsDone += 1
            screen, reward, isDone, info = ENVIRONMENT.step(action)
            pureRewardPerGame += reward
            reward = calculateRewardWithInfoGiven(reward, info, isDone)
            stateHolder.pushScreen(screen)
            
            
            if info['ale.lives'] < currentLifes:
                currentLifes -=1
                gameMemory.pushScreenActionReward(screen, action, reward, True)
            else:
                gameMemory.pushScreenActionReward(screen, action, reward, isDone)
                
            totalDisqountedReward = reward + totalDisqountedReward * DISCOUNT_FACTOR
            
            #progressBar.update(5 - info['ale.lives'])
            #performing neural network training on our replayMemory
            statesBatch, actionsBatch, nextStatesBatch, rewardsBatch, terminalMask = gameMemory.getBatch()
            currentQValues = keyNet(statesBatch).gather(1, actionsBatch.unsqueeze(1))
            nextQValues = torch.zeros(BATCH_SIZE, device = DEVICE)
            nextQValues = helperNet(nextStatesBatch).max(1)[0].detach()
            nextQValues[terminalMask == 1] = 0
            expectedQValues = rewardsBatch + nextQValues * DISCOUNT_FACTOR
            expectedQValues = torch.tensor(expectedQValues).unsqueeze(1).to(DEVICE)
            loss = F.smooth_l1_loss(currentQValues, expectedQValues)
            
            optimizer.zero_grad()
            loss.backward()
            for param in keyNet.parameters():
                param.grad.data.clamp_(-1, 1)
            optimizer.step()
        
        if e % 1 == 0:
            print("episode -", e)
            print("totalDisqountedReward = %f" % (totalDisqountedReward))
            print("pureRewardPerGame  = %f" % (pureRewardPerGame ))
            print("GameMemorySize = %d" % (len(gameMemory)))
            print()
        saveToFile(keyNet, "QNet.pkl")
        saveToFile(gameMemory, "gameMemory.pkl")
        if e % HELPER_UPDATE == 0:
            helperNet.load_state_dict(keyNet.state_dict())
            helperNet.eval()



mainCycle()



started learning
episode - 0
totalDisqountedReward = 0.000000
pureRewardPerGame  = 0.000000
GameMemorySize = 50000

episode - 1
totalDisqountedReward = 2.616738
pureRewardPerGame  = 1.000000
GameMemorySize = 50000

episode - 2
totalDisqountedReward = 0.912400
pureRewardPerGame  = 2.000000
GameMemorySize = 50000

episode - 3
totalDisqountedReward = 0.000000
pureRewardPerGame  = 4.000000
GameMemorySize = 50000

episode - 4
totalDisqountedReward = 0.484887
pureRewardPerGame  = 3.000000
GameMemorySize = 50000

episode - 5
totalDisqountedReward = 0.000000
pureRewardPerGame  = 0.000000
GameMemorySize = 50000

episode - 6
totalDisqountedReward = 0.665159
pureRewardPerGame  = 2.000000
GameMemorySize = 50000

episode - 7
totalDisqountedReward = 0.000000
pureRewardPerGame  = 0.000000
GameMemorySize = 50000

episode - 8
totalDisqountedReward = 0.000000
pureRewardPerGame  = 0.000000
GameMemorySize = 50000

episode - 9
totalDisqountedReward = 0.000977
pureRewardPerGame  = 2.000000
GameMemorySize = 